## Load data

In [1]:
import sys
from pathlib import Path

# Path(__file__).parent.parent nếu là file .py, 
# nhưng trong Notebook ta dùng Path.cwd()
root = Path.cwd().parent 
sys.path.append(str(root))

In [2]:
from GP_Solution.load_data import load_data
from GP_Solution.problem_structures import Truck, Drone

In [3]:
json_file_path = Path('..') /'..'/ 'data' / 'WithTimeWindows' / '100.10.1.json'

print("Đường dẫn tuyệt đối:", json_file_path.resolve())

try:
    pro = load_data(json_file_path)
    print("Đọc thành công!")
    print(f"Số lượng requests: {len(pro.requests)}")
    print(f"Số lượng vehicles: {len(pro.vehicles)}")

    # Kiểm tra request đầu tiên
    first_req = pro.requests[0]
    print(f"Request 0 location: {first_req.location}, Demand: {first_req.demand}")

    found_truck = False
    found_drone = False

    for veh in pro.vehicles:
        if found_truck and found_drone:
            break
        if not found_truck and isinstance(veh, Truck):
            truck = veh
            found_truck = True
        elif not found_drone and isinstance(veh, Drone):
            drone = veh
            found_drone = True

    print(f"Truck velocity: {truck.capacity} | vehicle_id: {truck.id}")
    print(f"Drone max range: {drone.max_range} | vehicle_id: {drone.id}")

except FileNotFoundError as e:
    print(f"Lỗi: {e}")
except Exception as e:
    print(f"Đã có lỗi xảy ra: {e}")


Đường dẫn tuyệt đối: D:\Project_all\prj3\data\WithTimeWindows\100.10.1.json
Đọc thành công!
Số lượng requests: 100
Số lượng vehicles: 8
Request 0 location: (3686.7818082186177, 4843.61036550108), Demand: 1.1093928409308884
Truck velocity: 400.0 | vehicle_id: 1
Drone max range: 700.0 | vehicle_id: 5


In [4]:
from GP_Solution.initalization import create_greedy_pop

In [5]:
pop = create_greedy_pop(50)

In [6]:
print(len(pop))

50


In [7]:
for i, ind in enumerate(pop):
    print(f"Individual {i}: {ind.r_tree.depth()} {ind.s_tree.depth()}")
    ind.to_string()

Individual 0: 2 2
R: (add RT3 RT1) | S: (min ST0 ST2)
Individual 1: 1 2
R: RT3 | S: (min ST0 ST2)
Individual 2: 2 1
R: (add RT1 RT3) | S: ST2
Individual 3: 3 1
R: (add RT3 (mul RT3 RT5)) | S: ST0
Individual 4: 2 2
R: (add RT0 RT3) | S: (min ST0 ST2)
Individual 5: 2 2
R: (add RT1 RT2) | S: (min ST0 ST2)
Individual 6: 1 2
R: RT3 | S: (div ST2 ST0)
Individual 7: 1 1
R: RT3 | S: ST2
Individual 8: 4 1
R: (add (sub (div RT3 RT3) RT3) RT2) | S: ST2
Individual 9: 5 5
R: (max RT3 (max (mul (add RT1 RT3) (sub RT3 RT3)) (max (add RT3 RT2) (add RT4 RT3)))) | S: (mul (add (add (add ST0 ST2) (mul ST1 ST0)) (min (sub ST2 ST0) (add ST2 ST3))) (min (mul (div ST0 ST3) (mul ST0 ST2)) (sub ST2 ST0)))
Individual 10: 5 1
R: (sub (mul (add (min RT3 RT3) (sub RT3 RT3)) RT3) (max RT3 (min (mul RT3 RT3) RT2))) | S: ST3
Individual 11: 1 1
R: RT5 | S: ST1
Individual 12: 5 5
R: (mul (max RT0 (max (sub RT3 RT3) (div RT2 RT3))) RT2) | S: (mul (sub ST2 (div (sub ST0 ST2) (div ST3 ST2))) ST1)
Individual 13: 5 5
R: (di

In [8]:
from GP_Solution.nsga2_algorithm import run_gphh_evolution

In [9]:
# pop_size = kwargs.get('pop_size', 50)
# c_rate = kwargs.get('c_rate', 0.8)
# m_rate = kwargs.get('m_rate', 0.2)
# max_depth = kwargs.get('max_dept', 6) # full depth for mutation, crossover
# tourn_s_size = kwargs.get('tourn_s_size', 4)
# max_gen = kwargs.get('max_generations', 20)

In [10]:
# --- CHẠY THỬ NGHIỆM ---

# 1. Load Problem
problem_instance = load_data(json_file_path)
print(f"Problem Loaded: {len(problem_instance.requests)} requests, {len(problem_instance.vehicles)} vehicles")

# 2. Chạy thuật toán GPHH
POP_SIZE = 200
GENS = 30
c_rate = 0.8
m_rate = 0.2
tourn_s_size = 4

final_pop, pareto, history = run_gphh_evolution(
    problem_instance, 
    pop_size=POP_SIZE, 
    max_gen=GENS, 
    max_depth=5,
    c_rate=c_rate,
    m_rate=m_rate,
    tourn_s_size=tourn_s_size,
)

print("\n=== FINAL PARETO FRONT ===")
print(f"Found {len(pareto)} non-dominated solutions.")

for i, ind in enumerate(pareto):
    close_time = problem_instance.depot_time_window[1]
    raw_makespan = (1.0 - ind.f2) * close_time
    
    print(f"\nSolution {i+1}:")
    print(f"  Served Ratio: {ind.f1:.2%} ({int(ind.f1*len(problem_instance.requests))} reqs)")
    print(f"  Makespan:     {raw_makespan:.1f}")
    print(f"  Rule R: {ind.r_tree.to_string()}")
    print(f"  Rule S: {ind.s_tree.to_string()}")

Problem Loaded: 100 requests, 8 vehicles
Gen   1 | Best Ratio: 0.810 | Best Time Score: 0.458
Gen   2 | Best Ratio: 0.810 | Best Time Score: 0.458
Gen   3 | Best Ratio: 0.820 | Best Time Score: 0.468
Gen   4 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen   5 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen   6 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen   7 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen   8 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen   9 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  10 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  11 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  12 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  13 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  14 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  15 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  16 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  17 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  18 | Best Ratio: 0.820 | Best Time Score: 0.473
Gen  